In [1]:
#@title Librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
from scipy.integrate import simpson, romberg, trapezoid

# Actividad 07: Integración

---
### Profesor: Juan Marcos Marín
### Nombre: Kevin Cortés Cordero
*Métodos computacionales 2024-II*

---

#1
* Implemente una función para el **método de integración de Romberg** definiendo un límite de tolerancia de 1e-8 y/o un máximo de iteraciones de 10.

* Encuentre la integral para

$$\int_0^{\pi/4} dx\, e^{3x}\cdot \sin(x)$$

* Imprima su resultado y compare los valores dados por `scipy.integrate.romberg`

* Finalmente, encuentre el valor del error, hallando el valor exacto usando `sympy`



In [ ]:
def integral_romberg(f, a, b, tol=1e-8, max_iter=10):
    R = np.zeros((max_iter, max_iter))
    h = b - a
    R[0, 0] = (h / 2) * (f(a) + f(b))

    for k in range(1, max_iter):
        h /= 2
        suma = sum(f(a + (2 * i - 1) * h) for i in range(1, 2**(k-1) + 1))
        R[k, 0] = 0.5 * R[k-1, 0] + h * suma

        for j in range(1, k + 1):
            R[k, j] = R[k, j-1] + (R[k, j-1] - R[k-1, j-1]) / (4**j - 1)

        if abs(R[k, k] - R[k-1, k-1]) < tol:
            return R[k, k]

    return R[max_iter-1, max_iter-1]

f = lambda x: np.exp(3*x) * np.sin(x)

integral_1 = integral_romberg(f, 0, np.pi/4)
print(f'El resultado con una función implementada es: {integral_1}')
integral_2 = romberg(f, 0, np.pi / 4)
print(f'El resultado usando scipy es: {integral_2}')

x = sp.symbols('x')
f_1 = sp.exp(3*x) * sp.sin(x)
integral_3 = sp.integrate(f_1, (x, 0, sp.pi/4))
integral_3_num = integral_3.evalf()
print(f'usando symbolab se obtiene: {integral_3_num}')
error = abs(integral_1 - integral_3_num)
print(f'El error es de: {error}')

El resultado con una función implementada es: 1.5920977078586134
El resultado usando scipy es: 1.592097707858613
usando symbolab se obtiene: 1.59209770785868
El error es de: 6.59472476627343E-14


<ipython-input-18-fa4825ef47a6>:23: DeprecationWarning: `scipy.integrate.romberg` is deprecated as of SciPy 1.12.0and will be removed in SciPy 1.15.0. Please use`scipy.integrate.quad` instead.
  integral_2 = romberg(f, 0, np.pi / 4)


#2

* Usando los *métodos trapezoidal compuesto*, *simpson 1/3* y de *medio punto* encuentre la siguiente integral,

$$\int_e^{1+e} dx\, \frac{1}{x\ln x}$$

* Luego, haga un estudio de la convergencia en términos del valor de $h$ o de los sub-intervalos de la función. ¿Cuál es mejor?


In [11]:
#Trapezoidal
def trapecio(f, a, b, n):

  h = (b - a) / n
  integral = 0.5 * (f(a) + f(b))
  for i in range(1, n):
    x_i = a + i * h
    integral += f(x_i)
  return integral * h, h

#Simpson
def simpson13(f, a, b, n):
    h = (b - a) / n
    integral = (f(a) + f(b))

    for i in range(1, n, 2):
      x_i = a + i * h
      integral += 4 * f(x_i)

    for i in range(2, n, 2):
      x_i = a + i * h
      integral += 2 * f(x_i)
    return integral * h/3, h

#Punto medio
def cuadratura_riemann(f, a, b, n):
  h = (b - a) / n
  integral = 0
  for i in range(n):
    x_i = a + i * h
    integral +=  f(x_i + h/2)
  return integral * h, h

f = lambda x: 1 / (x* np.log(x))

integral_trapezoide, convergencia1 = trapecio(f, np.e, np.e +1, 1000 )
integral_simpson, convergencia2 = simpson13(f, np.e, np.e +1, 1000)
integral_cuadratura, convergencia3 = cuadratura_riemann(f, np.e, np.e +1, 1000)
print(f'''
Método de trapezoide: {integral_trapezoide} con h= {convergencia1}
Método de Simpson: {integral_simpson} con h= {convergencia2}
Método de punto medio: {integral_cuadratura} con h= {convergencia3}
''')


Método de trapezoide: 0.2725138949739035 con h= 0.001
Método de Simpson: 0.27251388050258657 con h= 0.001
Método de punto medio: 0.2725138732669239 con h= 0.001



#3
Usando la siguiente función:



```python
def gauss_quad_standard(func, n):
    """
    Calcula la integral de una función en el intervalo [-1, 1]
    utilizando cuadratura gaussiana.

    Parameters:
    - func: La función a integrar.
    - n: Número de puntos para la cuadratura (grado del polinomio de Legendre).

    Returns:
    - Aproximación de la integral.
    """
    # Obtener raíces y pesos del polinomio de Legendre
    x, w = roots_legendre(n)

    # Evaluar la suma ponderada
    integral = np.sum(w * func(x))
    return integral
```

Modifique la función `gauss_quad_standard` de forma tal que no este restringida para $[-1,1]$ sino para cualquier intervalo $[a,b]$. Luego, encuentre la integral del *punto 2*.





In [14]:
from scipy.special import roots_legendre

def gauss_quad(func, a, b, n):

    # Obtener raíces y pesos del polinomio de Legendre
    x, w = roots_legendre(n)

    # Transformación de los puntos de [-1,1] a [a,b]
    t = 0.5 * (b - a) * x + 0.5 * (b + a)

    # Evaluar la suma ponderada con el factor de cambio de variable
    integral = np.sum(w * func(t)) * (b - a) / 2
    return integral

f = lambda x: 1 / (x*np.log(x))

integral_gauss = gauss_quad(f,np.e, np.e+1,1000)

print(f'La integral es: {integral_gauss}')


La integral es: 0.27251388050258063


#4

Encuentra todas las raices para los polinomios de grado 3 y 4 de **Legendre** usando el Método de la Secante y Newton-Raphson.



```python
import sympy as sp
x = sp.Symbol('x')

# Polinomio de Legendre de grado n
Pn = sp.legendre(n, x)

```

y calcule los pesos $w_i$ de la cuadratura mediante la fórmula:
   $$
   w_i = \frac{2}{(1 - x_i^2) \left[P_n'(x_i)\right]^2},
   $$
   donde $P_n'(x)$ es la derivada del polinomio de Legendre $P_n(x)$.
